San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [87]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import panel as pn
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv

# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')

In [88]:
import hvplot.pandas

In [89]:
# Read the Mapbox API key
load_dotenv(dotenv_path="mapbox.env")
map_box_api = os.getenv("map_box_api")

# Set the Mapbox API
px.set_mapbox_access_token(map_box_api)

Import Data

In [90]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. These functions will convert the plot object to a Panel pane.Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.Return a Panel pane object from each function that can be used to build the dashboard.Note: Remove any .show() lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [91]:
# Define Panel Visualization Functions
def housing_units_per_year():
    avg_house_per_year = sfo_data['housing_units'].groupby(by='year').mean().reset_index()
    fig_housing_units = avg_house_per_year.hvplot.bar(ylim=[370000, 386000],
                                                      title='Housing Units in San Francisco from 2010 to 2016')
    avg_house_per_year_plot.set_xlabel("Year")
    avg_house_per_year_plot.set_ylabel("Housing Units")
    return fig_housing_units

In [92]:
def average_gross_rent():
    avg_costs = sfo_data[['sale_price_sqr_foot','gross_rent']].groupby(by='year').mean()
    avg_gross_rent = avg_costs['gross_rent'].hvplot()
    return avg_gross_rent

In [93]:
def average_sales_price():
    avg_costs = sfo_data[['sale_price_sqr_foot','gross_rent']].groupby(by='year').mean()
    avg_sales_price = avg_costs['sale_price_sqr_foot'].hvplot()
    return avg_sales_price

In [94]:
def average_price_by_neighborhood():
    yr_neigh_grp = sfo_data.groupby(['year', 'neighborhood']).mean()
    yr_neigh_grp = yr_neigh_grp.reset_index()
    avg_price_by_neigh = yr_neigh_grp.hvplot.line(x='year', y='sale_price_sqr_foot', ylabel='Avg. Sale Price per Square Foot', groupby='neighborhood')
    return avg_price_by_neigh

In [95]:
def top_most_expensive_neighborhoods():
    top_10 = sfo_data.groupby(['neighborhood']).mean().sort_values(by='sale_price_sqr_foot', ascending=False)[:10]
    top_10.hvplot.bar(x='neighborhood', y='sale_price_sqr_foot', title='Top 10 Expensive SFO Neighborhoods', width=1200)
    return top_10_plot

In [96]:
def parallel_coordinates():
    top_10 = sfo_data.groupby(['neighborhood']).mean().sort_values(by='sale_price_sqr_foot', ascending=False)[:10]
    
    top_10_coord_plot = px.parallel_coordinates(
    top_10,
    dimensions=["sale_price_sqr_foot", "housing_units", "gross_rent"],
    color="gross_rent",
    labels={
        "sale_price_sqr_foot": "Price per Square Foot",
        "housing_units": "# Of Housing Units",
        "prop_size": "Property Size",
        "gross_rent": "Gross Rent",
    }
)
    return top_10_coord_plot

In [97]:
def parallel_categories():
    top_10 = sfo_data.groupby(['neighborhood']).mean().sort_values(by='sale_price_sqr_foot', ascending=False)[:10]
    
    top_10_cat_plot = px.parallel_categories(
    top_10.reset_index(),
    dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
    color="gross_rent",
    #color_continuous_scale=px.colors.sequential.Inferno,
    labels={
        "neighborhood": "Neighborhoods",
        "sale_price_sqr_foot": "Price per Square Foot",
        "housing_units": "# Of Housing Units",
        "prop_size": "Property Size",
        "gross_rent": "Gross Rent",
    }
)
    return top_10_cat_plot

In [134]:
def neighborhood_map():
    sfo_avgs = sfo_data.groupby(['neighborhood']).mean().reset_index()
    sfo_avgs = pd.concat([sfo_avgs, df_neighborhood_locations], axis='columns', join='inner')
    
    neigh_location = px.scatter_mapbox(
    sfo_avgs,
    lat="Lat",
    lon="Lon",
    size="sale_price_sqr_foot",
    color="gross_rent",
    color_continuous_scale=px.colors.cyclical.IceFire,
    title="Neighborhood Locations in San Francisco",
    zoom=10,
    height=750
    
)
    return neigh_location

Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [144]:
Intro = pn.Column(
    ("""This Dashboard shows property data for San Francisco from 2010 to 2016. 
    You can navigate through the various tabs above to explore more details of the analysis of San Francisco real estate market. """),
    ("## Average Sale Price Per Square Foot and Gross Rent in San Francisco"),
    neighborhood_map()
)
    
rental_analysis = pn.Column(
    ("## San Francisco Housing Rental Analysis - 2010 to 2016"),
    housing_units_per_year(),
    average_gross_rent(),
    average_sales_price()
)

neigh_analysis = pn.Column(
    ("## San Francisco Top 10 Most Expensive Neighborhood Analysis - 2010 to 2016"),
    average_price_by_neighborhood(),
    top_most_expensive_neighborhoods()
)

parallel_plot = pn.Column(
    ("## Parallel Coordinates and Parallel Categories Analysis"),
    parallel_coordinates(),
    parallel_categories()
)

In [145]:
panel = pn.Tabs(
    ("About", Intro),
    ("Rental Analysis", rental_analysis),
    ("Neighborhood Analysis", neigh_analysis),
    ("Parallel Plots Analysis",parallel_plot)
)

Serve the Panel Dashboard

In [146]:
panel.servable().show()

Launching server at http://localhost:61246
